In [1]:
import base64
import requests
import json
import pandas as pd
#import pprint
#from bs4 import BeautifulSoup as BS

In [2]:
username = open('../data/rav_user.txt').read()
password = open('../data/rav_pass.txt').read()

How to retrieve the details I need from pattern listings:

- Use code from test notebook to retrieve 10,000 patterns sorted by popularity
- Using id column, write for loop (I think, maybe something else is better) to download patter details (an example using R is on RavelRy github)

retrieve pattern listings:

In [ ]:
# top 1000 patterns sorted by most projects

def get_user_info(query, username, password):
    print(query)
    print(username)
    print(password)
    print('https://api.ravelry.com/patterns/search.json')
    res = requests.get('https://api.ravelry.com/patterns/search.json?sort=projects&page_size=1000&page=1'.format(query),
                       auth=requests.auth.HTTPBasicAuth(username, password))
    return res

In [ ]:
res = get_user_info('', username, password)

In [ ]:
res

In [ ]:
result = json.loads(res.content)

In [ ]:
result['patterns']

In [ ]:
patterns_df = pd.DataFrame(result['patterns'])

In [ ]:
patterns_df.tail()

In [ ]:
result['paginator']

In [ ]:
result['patterns'][0]

In [ ]:
id_list = patterns_df.id.tolist()

In [ ]:
# manual testing to see where the number of projects declines - very long tail!

def get_user_info(query, username, password):
    print(query)
    print(username)
    print(password)
    print('https://api.ravelry.com/patterns/search.json')
    res = requests.get('https://api.ravelry.com/patterns/search.json?query={}&sort=projects&page_size=1000&page=80'.format(query),
                       auth=requests.auth.HTTPBasicAuth(username, password))
    return res

In [ ]:
res = get_user_info('', username, password)

In [ ]:
result = json.loads(res.content)
result['patterns']
patterns2_df = pd.DataFrame(result['patterns'])
patterns2_df.tail()

thinking through code to retrieve results

- pattern ids in list
- create empty df
- define function to pass in username and password
- url to retrieve a single result: base url + patternid as str + .json
- for loop to iterate through id_list
- append each result to df

Initially I was thinking to get top 10,000 but that might not make sense considering the long tail (see notes). Should I cut off at 100 projects? Justification would be that 100 projects is enough to ensure that the pattern appeals to a good number of people. My purpose for this part of the analysis is to determine what type of patterns people like and what common characteristics there are - yarn weights, category, possibly amount of yarn required.

If I take the top 30,000 patterns, I'll need to have a for loop and defined function to iterate over the pages - can I pull the ids straight out into a list from the json in the loop?

Then use list of ids in a for loop to iterate through pattern details and pull results into a dataframe (as in R example in RavelRy project).

notes:

10,000th pattern has 241 projects, while 1st pattern has 23.6k.

12,000th pattern has 206 projects. 15,000th project has 169. I'm actually a little surprised by the length of the tail.

20,000th pattern has 130 projects. 30,000th has 88. 40,000th has 67. 60,000th has 43. 80,000th has 31.

In [3]:
def get_user_info(username, password):
    pattid_list = []
    for page in range(1, 31):
        print(page)
        res = requests.get('https://api.ravelry.com/patterns/search.json?sort=projects&page_size=1000&page=' + str(page),
                       auth=requests.auth.HTTPBasicAuth(username, password))
        result = json.loads(res.content)
        patterns_df = pd.DataFrame(result['patterns'])
        print(patterns_df.shape)
        pattid_list.append(patterns_df.id.tolist())
    return pattid_list


In [4]:
# returns pattern ids as integers, list of lists (a list for each page of results)
pattid_list = get_user_info(username, password)

1
(1000, 9)
2
(1000, 9)
3
(1000, 9)
4
(1000, 9)
5
(1000, 9)
6
(1000, 9)
7
(1000, 9)
8
(1000, 9)
9
(1000, 9)
10
(1000, 9)
11
(1000, 9)
12
(1000, 9)
13
(1000, 9)
14
(1000, 9)
15
(1000, 9)
16
(1000, 9)
17
(1000, 9)
18
(1000, 9)
19
(1000, 9)
20
(1000, 9)
21
(1000, 9)
22
(1000, 9)
23
(1000, 9)
24
(1000, 9)
25
(1000, 9)
26
(1000, 9)
27
(1000, 9)
28
(1000, 9)
29
(1000, 9)
30
(1000, 9)


In [ ]:
print(pattid_list)

In [5]:
# to flatten list, and change integers into strings

pattid_flat_list = [str(patt_id) for sublist in pattid_list for patt_id in sublist]

In [ ]:
len(pattid_flat_list)

In [ ]:
pattid_flat_list

In [ ]:
# method performed on a string, links each item in the iterable (in this case linked by + since that's in '')

'+'.join(pattid_flat_list)

In [ ]:
pattid_list[0]

In [ ]:
# turn ids contained in the first list into strings

pattid_list_str = [str(patt_id) for patt_id in pattid_list[0]]
pattid_list_str

In [ ]:
res = requests.get('https://api.ravelry.com/patterns.json?ids=' + '+'.join(pattid_list_str[:200]), auth=requests.auth.HTTPBasicAuth(username, password))

In [ ]:
res.content

In [ ]:
res = requests.get('https://api.ravelry.com/patterns.json?ids=' + '+'.join(pattid_flat_list), auth=requests.auth.HTTPBasicAuth(username, password))

In [ ]:
res.content

In [ ]:
# not as expected, I'm not sure why - come back to the problem when my brain is fresher

def get_details(username, password, pattids_list):
    patt_details_list = []
    for ids in pattids_list:
        print(ids)
        res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                           '+'.join(pattid_list_str[:2]), auth=requests.auth.HTTPBasicAuth(username, password))
        result = json.loads(res.content)
        pattdetail_df = pd.DataFrame(result['patterns'])
        print(pattdetail_df.shape)
        patt_details_list.append(pattdetail_df)
    return pd.concat(patt_details_list)
        


In [ ]:
pattdetails_df = get_details(username, password, pattid_list_str)

In [ ]:
# start here, don't do for loop inside function, break list of ids into managable chunks and put that into the request.
# also, the data is wide instead of long (each pattern's data is in a column of 50 rows instead of 1 row across)
# maybe if I go back to the function where I'm pulling the ids, make the page size smaller, then I could pass in
# each list?


res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                   '+'.join(pattid_list_str[:200]), auth=requests.auth.HTTPBasicAuth(username, password))


Plan of attack

First, write chunk of code to return pattern details for a single id. Should be made into a dataframe and pivoted so that information is in a row instead of a column. (melt, wide_to_long, transpose (pd.t))

Second, expand the code to take multiple ids. Not quite sure how to do this, but maybe once I accomplish the first goal I'll have a better idea. After talking to M&M, wrote function and for loop to iterate through a test list of 10 ids pulled from the flattened/string-ified list. Continued to use transform to switch rows and columns, and appended each result to a dataframe.

Third, since I don't want to call the API 30,000 times to get the pattern details, can I chunk the flattened id list into groups of maybe 200 and then use the code from the second step on that list? Would call the API 150 time which is much more reasonable. Mahesh hinted that I might use the range function, although that was before I expanded into a third step. Still, definitely worth looking in to.


In [6]:
# first part of plan accomplished! Glad I was at the point of having a df when I talked with M&M - Mary suggested
# transpose from pandas which accomplished the final part very easily. Magic!

first = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                     str(211562), auth=requests.auth.HTTPBasicAuth(username, password))
first_result = json.loads(first.content)
first_detail_df = pd.DataFrame(first_result['patterns']).T


In [7]:
first_detail_df

,comments_count,craft,created_at,currency,currency_symbol,difficulty_average,difficulty_count,download_location,downloadable,favorites_count,...,row_gauge,sizes_available,updated_at,url,volumes_in_library,yardage,yardage_description,yardage_max,yarn_weight,yarn_weight_description
211562,146,"{'id': 2, 'name': 'Knitting', 'permalink': 'kn...",2010/11/15 07:59:40 -0500,EUR,€,1.76018,9528,"{'type': 'ravelry', 'free': False, 'url': 'htt...",True,49515,...,54,one,2019/10/28 12:20:59 -0400,,None,569,569 - 574 yards,574,"{'crochet_gauge': '', 'id': 5, 'knit_gauge': '...",Fingering (14 wpi)


In [8]:
# second part done, successful function to iterate through lists and append each result to df.
# Also reset index and renamed, since pattern id was put in the index position.

second_test_id_list = ['211562', '130787', '605', '169260', '29', '124400', '573', '426231', '418518', '195']

def second_get_details(username, password, test_id_list):
    second_details_df = pd.DataFrame()
    for ids in second_test_id_list:
        second = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                     ids, auth=requests.auth.HTTPBasicAuth(username, password))
        second_result = json.loads(second.content)
        detail_df = pd.DataFrame(second_result['patterns']).T
        second_details_df = second_details_df.append(detail_df)
        print(second_details_df.shape)
    return second_details_df.reset_index().rename(columns = {'index': 'patt_id'})


In [9]:
second_test_df = second_get_details(username, password, second_test_id_list)

(1, 50)
(2, 50)
(3, 50)
(4, 50)
(5, 50)
(6, 50)
(7, 50)
(8, 50)
(9, 50)
(10, 50)


In [10]:
second_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   patt_id                  10 non-null     object
 1   comments_count           10 non-null     object
 2   craft                    10 non-null     object
 3   created_at               10 non-null     object
 4   currency                 5 non-null      object
 5   currency_symbol          5 non-null      object
 6   difficulty_average       10 non-null     object
 7   difficulty_count         10 non-null     object
 8   download_location        9 non-null      object
 9   downloadable             10 non-null     object
 10  favorites_count          10 non-null     object
 11  free                     10 non-null     object
 12  gauge                    9 non-null      object
 13  gauge_description        10 non-null     object
 14  gauge_divisor            10 non-null     obje

In [11]:
# to divide pattid_flat_list into chunks of 200

# How many elements each list should have  
n = 200
   
# using list comprehension  
pattid_chunk_list = [pattid_flat_list[i:i + n] for i in range(0, len(pattid_flat_list), n)]  
len(pattid_chunk_list)


150

In [14]:
# code for final part of plan

def get_details(username, password, id_list):
    details_df = pd.DataFrame()
    for ids in pattid_chunk_list:
        res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                           '+'.join(ids), auth=requests.auth.HTTPBasicAuth(username, password))
        try:        
            result = json.loads(res.content)
            detail_df = pd.DataFrame(result['patterns']).T
            details_df = details_df.append(detail_df)
            print(details_df.shape)
        except:
            print(res.content)
            continue
    return details_df.reset_index().rename(columns = {'index': 'patt_id'})


In [15]:
patt_details_df = get_details(username, password, pattid_chunk_list)

(200, 50)
(400, 50)
(600, 50)
(800, 50)
(1000, 50)
(1200, 50)
(1400, 50)
(1600, 50)
(1800, 50)
b' '
(2000, 50)
(2200, 50)
(2400, 50)
(2600, 50)
(2800, 50)
(3000, 50)
(3200, 50)
(3400, 50)
(3600, 50)
(3800, 50)
(4000, 50)
(4200, 50)
(4400, 50)
(4600, 50)
(4800, 50)
(5000, 50)
(5200, 50)
(5400, 50)
(5600, 50)
(5800, 50)
(6000, 50)
(6200, 50)
(6400, 50)
(6600, 50)
(6800, 50)
(7000, 50)
(7200, 50)
(7400, 50)
(7600, 50)
(7800, 50)
(8000, 50)
(8200, 50)
(8400, 50)
(8600, 50)
(8800, 50)
(9000, 50)
(9200, 50)
(9400, 50)
(9600, 50)
(9800, 50)
(10000, 50)
(10200, 50)
(10400, 50)
(10600, 50)
(10800, 50)
(11000, 50)
(11200, 50)
(11400, 50)
(11600, 50)
(11800, 50)
(12000, 50)
(12200, 50)
(12400, 50)
(12600, 50)
(12800, 50)
(13000, 50)
(13200, 50)
(13400, 50)
(13600, 50)
(13800, 50)
(14000, 50)
(14200, 50)
(14400, 50)
(14600, 50)
(14800, 50)
(15000, 50)
(15200, 50)
(15400, 50)
(15600, 50)
(15800, 50)
(16000, 50)
(16200, 50)
(16400, 50)
(16600, 50)
(16800, 50)
(17000, 50)
(17200, 50)
(17400, 50)
(176

In [19]:
# to deal with the bad value in the chunk at index 9
# append the result to patt_details_df to get the complete dataset I want to end up with
# every line resulted in "b' '" indicating a problem, dataframe was empty at the end

def get_problem_details(username, password, id_list):
    problem_df = pd.DataFrame()
    for ids in id_list:
        res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                           '+'.join(ids), auth=requests.auth.HTTPBasicAuth(username, password))
        try:        
            result = json.loads(res.content)
            detail_df = pd.DataFrame(result['patterns']).T
            problem_df = problem_df.append(detail_df)
            print(problem_df.shape)
        except:
            print(res.content)
            continue
    return problem_df.reset_index().rename(columns = {'index': 'patt_id'})


In [20]:
bad_json_df = get_problem_details(username, password, pattid_chunk_list[9])

b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '
b' '


In [24]:
# to deal with the bad value in the chunk at index 9
# append the result to patt_details_df to get the complete dataset I want to end up with
# code from second step of testing; using index number of problem chunk, pass that in for loop to run on each id
# individually
# incorporate try/except from previous function to skip over the problem row

def get_problem_details(username, password, id_list):
    problem_df = pd.DataFrame()
    for ids in id_list:
        res = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                           ids, auth=requests.auth.HTTPBasicAuth(username, password))
        try:
            result = json.loads(res.content)
            detail_df = pd.DataFrame(result['patterns']).T
            problem_df = problem_df.append(detail_df)
            print(problem_df.shape)
        except:
            print(res.content)
            continue
    return problem_df.reset_index().rename(columns = {'index': 'patt_id'})


In [25]:
bad_json_df = get_problem_details(username, password, pattid_chunk_list[9])

(1, 50)
(2, 50)
(3, 50)
(4, 50)
(5, 50)
(6, 50)
(7, 50)
(8, 50)
(9, 50)
(10, 50)
(11, 50)
(12, 50)
(13, 50)
(14, 50)
(15, 50)
(16, 50)
(17, 50)
(18, 50)
(19, 50)
(20, 50)
(21, 50)
(22, 50)
(23, 50)
(24, 50)
(25, 50)
(26, 50)
(27, 50)
(28, 50)
(29, 50)
(30, 50)
(31, 50)
(32, 50)
(33, 50)
(34, 50)
(35, 50)
(36, 50)
(37, 50)
(38, 50)
(39, 50)
(40, 50)
(41, 50)
(42, 50)
(43, 50)
(44, 50)
(45, 50)
(46, 50)
(47, 50)
(48, 50)
(49, 50)
(50, 50)
(51, 50)
(52, 50)
(53, 50)
(54, 50)
(55, 50)
(56, 50)
(57, 50)
(58, 50)
(59, 50)
(60, 50)
(61, 50)
(62, 50)
(63, 50)
(64, 50)
(65, 50)
(66, 50)
(67, 50)
(68, 50)
(69, 50)
(70, 50)
(71, 50)
(72, 50)
(73, 50)
(74, 50)
(75, 50)
(76, 50)
(77, 50)
(78, 50)
(79, 50)
(80, 50)
(81, 50)
(82, 50)
(83, 50)
(84, 50)
(85, 50)
(86, 50)
(87, 50)
(88, 50)
(89, 50)
(90, 50)
(91, 50)
(92, 50)
(93, 50)
(94, 50)
(95, 50)
(96, 50)
(97, 50)
(98, 50)
(99, 50)
(100, 50)
(101, 50)
(102, 50)
(103, 50)
(104, 50)
(105, 50)
(106, 50)
(107, 50)
(108, 50)
(109, 50)
(110, 50)
(111, 50

In [27]:
problem = pattid_chunk_list[9]
problem[147]

'20'

In [29]:
# repurposing code from first test to see if the problem id can be pulled individually
# confirmed that this is definitely the problem id and it cannot be called through API

prob_id = requests.get('https://api.ravelry.com/patterns.json?ids=' +
                     str(20), auth=requests.auth.HTTPBasicAuth(username, password))
prob_id_result = json.loads(prob_id.content)
prob_id_detail_df = pd.DataFrame(prob_id_result['patterns']).T
prob_id_detail_df = prob_id_detail_df.reset_index().rename(columns = {'index': 'patt_id'})


JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [31]:
# append bad_json_df to patt_details_df for complete pattern details dataset

patt_details_df = patt_details_df.append(bad_json_df).reset_index()
patt_details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30198 entries, 0 to 30197
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    30198 non-null  int64 
 1   patt_id                  30198 non-null  object
 2   comments_count           30198 non-null  object
 3   created_at               30198 non-null  object
 4   currency                 21638 non-null  object
 5   difficulty_average       30198 non-null  object
 6   difficulty_count         30175 non-null  object
 7   downloadable             30197 non-null  object
 8   favorites_count          30198 non-null  object
 9   free                     30197 non-null  object
 10  gauge                    21856 non-null  object
 11  gauge_divisor            23684 non-null  object
 12  gauge_pattern            30197 non-null  object
 13  generally_available      30198 non-null  object
 14  id                       30198 non-nul

In [34]:
patt_details_df.to_csv('../data/df_pattdetails.csv')